In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.0 
✔ readr   2.1.3      ✔ forcats 0.5.1 
Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Warning message:
“package ‘tibble’ was built under R version 4.1.2”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidym

## Team Factor Calculations
Data from baseball savant and razzball is used to calculate "Team factor". Team factor consists of the average fantasy points per game for each team from 2022 standardized to a normal distribution. Abnormalities in the usual standardization of a Normal(0,1) distribution are in place in order to prevent extreme transformation on the final prediction, as the standard deviation is decreased significantly and the mean is centered at 1. Team factor is simply meant to allow for a slight influence on the team (surrounding players and park) that each player is on for the final predictions. Changes to the Blue Jays, Tigers and Mets ballpark dimensions are accounted for with an approximate estimate on how these changes will impact offensive production. 

In [2]:
data <- read_csv("data/2022data.csv")
head(data)

New names:
• `` -> `...12`
Rows: 469 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): last_name, first_name
dbl (9): player_id, year, b_strikeout, b_walk, b_rbi, b_total_bases, r_total...
lgl (1): ...12

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,b_strikeout,b_walk,b_rbi,b_total_bases,r_total_stolen_base,b_game,r_run,...12
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Pujols,Albert,405395,2022,55,28,68,169,1,109,42,NA
Cabrera,Miguel,408234,2022,101,28,43,126,1,112,25,NA
Molina,Yadier,425877,2022,40,5,24,79,2,78,19,NA
Cano,Robinson,429664,2022,25,4,4,19,0,33,5,NA
Suzuki,Kurt,435559,2022,29,15,15,41,0,51,10,NA
Cruz Jr.,Nelson,443558,2022,119,49,64,151,4,124,50,NA


In [3]:
selected <- data %>% select(-"player_id", -"...12")
head(selected)

last_name,first_name,year,b_strikeout,b_walk,b_rbi,b_total_bases,r_total_stolen_base,b_game,r_run
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Pujols,Albert,2022,55,28,68,169,1,109,42
Cabrera,Miguel,2022,101,28,43,126,1,112,25
Molina,Yadier,2022,40,5,24,79,2,78,19
Cano,Robinson,2022,25,4,4,19,0,33,5
Suzuki,Kurt,2022,29,15,15,41,0,51,10
Cruz Jr.,Nelson,2022,119,49,64,151,4,124,50


In [4]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>% 
rename("K" = "b_strikeout", "BB" = "b_walk", "RBI" = "b_rbi", "TB" = "b_total_bases", "SB" = "r_total_stolen_base", 
           "R" = "r_run", "G" = "b_game") %>% mutate(Fpointsg = (BB + RBI + TB + SB + R - K)/G)

head(names)

player,year,K,BB,RBI,TB,SB,G,R,Fpointsg
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Pujols, Albert",2022,55,28,68,169,1,109,42,2.3211009
"Cabrera, Miguel",2022,101,28,43,126,1,112,25,1.0892857
"Molina, Yadier",2022,40,5,24,79,2,78,19,1.1410256
"Cano, Robinson",2022,25,4,4,19,0,33,5,0.2121212
"Suzuki, Kurt",2022,29,15,15,41,0,51,10,1.0196078
"Cruz Jr., Nelson",2022,119,49,64,151,4,124,50,1.6048387


In [5]:
teamdata <- read_csv("data/Razzpreds.csv") %>% select(Name, Team)
head(teamdata)

Rows: 743 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (5): Name, Team, Bats, ESPN, YAHOO
dbl (23): #, G, PA, AB, R, HR, RBI, SB, H, 1B, 2B, 3B, TB, SO, BB, HBP, SF, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Name,Team
<chr>,<chr>
Juan Soto,SD
Jose Altuve,HOU
Mookie Betts,LAD
Trea Turner,PHI
Freddie Freeman,LAD
Bo Bichette,TOR


In [6]:
names1 <- extract(teamdata,Name,c("First_Name","Last_Name"), "([^ ]+) (.*)") 
head(names1)

First_Name,Last_Name,Team
<chr>,<chr>,<chr>
Juan,Soto,SD
Jose,Altuve,HOU
Mookie,Betts,LAD
Trea,Turner,PHI
Freddie,Freeman,LAD
Bo,Bichette,TOR


In [7]:
real <- names1 %>% unite("player", Last_Name:First_Name, sep = ", ", remove = T)
head(real)

player,Team
<chr>,<chr>
"Soto, Juan",SD
"Altuve, Jose",HOU
"Betts, Mookie",LAD
"Turner, Trea",PHI
"Freeman, Freddie",LAD
"Bichette, Bo",TOR


In [8]:
write_csv(real, "data/playertm.csv")

In [9]:
full <- right_join(real, names, by = 'player') %>% filter(Team != "NA" & Team != "FA")
head(full)

player,Team,year,K,BB,RBI,TB,SB,G,R,Fpointsg
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Soto, Juan",SD,2022,96,135,62,237,6,153,93,2.856209
"Altuve, Jose",HOU,2022,87,66,57,281,18,141,103,3.106383
"Betts, Mookie",LAD,2022,104,55,82,305,12,142,117,3.288732
"Turner, Trea",PHI,2022,131,45,100,304,27,160,101,2.787500
"Freeman, Freddie",LAD,2022,102,84,100,313,13,159,117,3.301887
"Bichette, Bo",TOR,2022,155,41,93,306,13,159,91,2.446541


In [10]:
## Calculating team averages (dividing by five to assist with scaling)
team_avg <- full %>% group_by(Team) %>% summarize(avg = mean(Fpointsg)/5)
head(team_avg)

Team,avg
<chr>,<dbl>
ARI,0.3016269
ATL,0.3421113
BAL,0.2927034
BOS,0.3238261
CHC,0.2872872
CHW,0.3063281


In [11]:
## Standardizing team average points to a N(1, sd^(1/100)) disribution
team_avg$standardized <- ((team_avg$avg - mean(team_avg$avg))/(sd(team_avg$avg))^(1/100)) + 1
team_avg

Team,avg,standardized
<chr>,<dbl>,<dbl>
ARI,0.3016269,0.9794853
ATL,0.3421113,1.0212407
BAL,0.2927034,0.9702816
BOS,0.3238261,1.0023814
CHC,0.2872872,0.9646953
CHW,0.3063281,0.9843341
CIN,0.2405591,0.9165001
CLE,0.3730269,1.0531270
COL,0.3410039,1.0200986


In [12]:
## Adding impacts from new park dimensions:
team_avg[10,]$standardized = team_avg[10,]$standardized + 0.03
team_avg[29,]$standardized = team_avg[29,]$standardized + 0.03
team_avg[18,]$standardized = team_avg[18,]$standardized + 0.015
team_avg

Team,avg,standardized
<chr>,<dbl>,<dbl>
ARI,0.3016269,0.9794853
ATL,0.3421113,1.0212407
BAL,0.2927034,0.9702816
BOS,0.3238261,1.0023814
CHC,0.2872872,0.9646953
CHW,0.3063281,0.9843341
CIN,0.2405591,0.9165001
CLE,0.3730269,1.0531270
COL,0.3410039,1.0200986


In [13]:
write_csv(team_avg, "data/teamfactor.csv")